***Important*** DO NOT CLEAR THE OUTPUT OF THIS NOTEBOOK AFTER EXECUTION!!!

In [1]:
# if the following command generates an error, you probably didn't enable 
# the cluster security option "Allow API access to all Google Cloud services"
# under Manage Security → Project Access when setting up the cluster
!gcloud dataproc clusters list --region us-central1

NAME          PLATFORM  WORKER_COUNT  PREEMPTIBLE_WORKER_COUNT  STATUS   ZONE           SCHEDULED_DELETE
cluster-d716  GCE       2                                       RUNNING  us-central1-a


# Imports & Setup

In [2]:
!pip install -q google-cloud-storage==1.43.0
!pip install -q graphframes

In [3]:
import pyspark
import sys
from collections import Counter, OrderedDict, defaultdict
import itertools
from itertools import islice, count, groupby
import pandas as pd
import os
import re
from operator import itemgetter
import nltk
from nltk.stem.porter import *
from nltk.corpus import stopwords
from time import time
from pathlib import Path
import pickle
import pandas as pd
from google.cloud import storage

import hashlib
def _hash(s):
    return hashlib.blake2b(bytes(s, encoding='utf8'), digest_size=5).hexdigest()

nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [4]:
# if nothing prints here you forgot to include the initialization script when starting the cluster
!ls -l /usr/lib/spark/jars/graph*

-rw-r--r-- 1 root root 247882 Jan  8 10:33 /usr/lib/spark/jars/graphframes-0.8.2-spark3.1-s_2.12.jar


In [5]:
from pyspark.sql import *
from pyspark.sql.functions import *
from pyspark import SparkContext, SparkConf, SparkFiles
from pyspark.sql import SQLContext
from graphframes import *

In [6]:
spark

In [7]:
# Put your bucket name below and make sure you can access it without an error
bucket_name = 'buckeybucketbuck' 
full_path = f"gs://{bucket_name}/"
paths=[]

client = storage.Client()
blobs = client.list_blobs(bucket_name)
for b in blobs:
    #add only test file
    paths.append(full_path+"WikiData/multistream1_preprocessed.parquet")
    break
    
    if b.name != 'graphframes.sh'and b.name !='WikiData/graphframes.sh' and b.name != 'WikiData/':
        paths.append(full_path+b.name)

In [8]:
paths

['gs://buckeybucketbuck/WikiData/multistream1_preprocessed.parquet']

***GCP setup is complete!*** If you got here without any errors you've earned 10 out of the 35 points of this part.

# Building an inverted index

Here, we read the entire corpus to an rdd, directly from Google Storage Bucket and use your code from Colab to construct an inverted index.

In [9]:
parquetFile = spark.read.parquet(*paths)

#limit to 1000 for faster testing
doc_text_pairs = parquetFile.limit(1000).select("text", "id").rdd
doc_len = doc_text_pairs
#added title and anchor pairs
doc_title_pairs = parquetFile.limit(1000).select("title", "id").rdd
doc_to_title = parquetFile.limit(1000).select("id", "title").rdd

We will count the number of pages to make sure we are looking at the entire corpus. The number of pages should be more than 6M

In [10]:
# Count number of wiki pages
parquetFile.count()

21084

Let's import the inverted index module. Note that you need to use the staff-provided version called `inverted_index_gcp.py`, which contains helper functions to writing and reading the posting files similar to the Colab version, but with writing done to a Google Cloud Storage bucket.

In [11]:
# if nothing prints here you forgot to upload the file inverted_index_gcp.py to the home dir
%cd -q /home/dataproc
!ls best_inverted_index_ever.py

best_inverted_index_ever.py


In [12]:
# adding our python module to the cluster
sc.addFile("/home/dataproc/best_inverted_index_ever.py")
sys.path.insert(0,SparkFiles.getRootDirectory())

In [13]:
from best_inverted_index_ever import InvertedIndex

**YOUR TASK (10 POINTS)**: Use your implementation of `word_count`, `reduce_word_counts`, `calculate_df`, and `partition_postings_and_write` functions from Colab to build an inverted index for all of English Wikipedia in under 2 hours.

A few notes: 
1. The number of corpus stopwords below is a bit bigger than the colab version since we are working on the whole corpus and not just on one file.
2. You need to slightly modify your implementation of  `partition_postings_and_write` because the signature of `InvertedIndex.write_a_posting_list` has changed and now includes an additional argument called `bucket_name` for the target bucket. See the module for more details.
3. You are not allowed to change any of the code not coming from Colab. 

In [21]:
english_stopwords = frozenset(stopwords.words('english'))
corpus_stopwords = ["category", "references", "also", "external", "links", 
                    "may", "first", "see", "history", "people", "one", "two", 
                    "part", "thumb", "including", "second", "following", 
                    "many", "however", "would", "became"]

all_stopwords = english_stopwords.union(corpus_stopwords)
RE_WORD = re.compile(r"""[\#\@\w](['\-]?\w){2,24}""", re.UNICODE)
stemmer = PorterStemmer()

NUM_BUCKETS = 124
def token2bucket_id(token):
  return int(_hash(token),16) % NUM_BUCKETS

# PLACE YOUR CODE HERE
def word_count(text, id,use_stemming=False):
  ''' Count the frequency of each word in `text` (tf) that is not included in 
  `all_stopwords` and return entries that will go into our posting lists. 
  Parameters:
  -----------
    text: str
      Text of one document
    id: int
      Document id
  Returns:
  --------
    List of tuples
      A list of (token, (doc_id, tf)) pairs 
      for example: [("Anarchism", (12, 5)), ...]
  '''
  tokens = []
  if use_stemming:
    tokens = [stemmer.stem(token.group()) for token in RE_WORD.finditer(text.lower())]
  else:
    tokens = [token.group() for token in RE_WORD.finditer(text.lower())]        
 
  # YOUR CODE HERE
  res = []
  c = Counter([t for t in tokens if t not in all_stopwords])
  for word,tf in c.items():
     res.append((word,(id,tf)))
  return res
def reduce_word_counts(unsorted_pl):
  ''' Returns a sorted posting list by wiki_id.
  Parameters:
  -----------
    unsorted_pl: list of tuples
      A list of (wiki_id, tf) tuples 
  Returns:
  --------
    list of tuples
      A sorted posting list.
  '''
  # YOUR CODE HERE
  return sorted(unsorted_pl,key=lambda x:x[0])
def calculate_df(postings):
  ''' Takes a posting list RDD and calculate the df for each token.
  Parameters:
  -----------
    postings: RDD
      An RDD where each element is a (token, posting_list) pair.
  Returns:
  --------
    RDD
      An RDD where each element is a (token, df) pair.
  '''
  # YOUR CODE HERE
  return postings.map(lambda x: (x[0],len(x[1])))
def partition_postings_and_write(postings,name):
  ''' A function that partitions the posting lists into buckets, writes out 
  all posting lists in a bucket to disk, and returns the posting locations for 
  each bucket. Partitioning should be done through the use of `token2bucket` 
  above. Writing to disk should use the function  `write_a_posting_list`, a 
  static method implemented in inverted_index_colab.py under the InvertedIndex 
  class. 
  Parameters:
  -----------
    postings: RDD
      An RDD where each item is a (w, posting_list) pair.
  Returns:
  --------
    RDD
      An RDD where each item is a posting locations dictionary for a bucket. The
      posting locations maintain a list for each word of file locations and 
      offsets its posting list was written to. See `write_a_posting_list` for 
      more details.
  '''
  # YOUR CODE HERE
  p = postings.map(lambda x:(token2bucket_id(x[0]),x)).groupByKey().mapValues(list)
  return p.map(lambda x: InvertedIndex.write_a_posting_list(x,'buckeybucketbuck',name))

In [15]:
def get_doc_len(text,docid):
    tokens = [token.group() for token in RE_WORD.finditer(text.lower())]
    res = []
    for term in tokens:
        if term not in all_stopwords:
            res.append(term)

    return (docid,len(res))

In [16]:
doclen = (doc_text_pairs.map(lambda x: get_doc_len(x[0], x[1]))).collectAsMap()
doclen

{12: 4064,
 25: 4125,
 39: 1843,
 290: 1000,
 303: 8088,
 305: 4298,
 307: 7928,
 308: 6030,
 309: 1015,
 316: 4619,
 324: 4339,
 330: 248,
 332: 233,
 334: 676,
 336: 3728,
 339: 4177,
 340: 299,
 344: 805,
 358: 6584,
 359: 2551,
 569: 4154,
 572: 632,
 573: 4895,
 579: 440,
 580: 457,
 586: 3370,
 590: 195,
 593: 3603,
 594: 9953,
 595: 5304,
 597: 2288,
 599: 3170,
 600: 5100,
 612: 726,
 615: 895,
 620: 3717,
 621: 6896,
 624: 7099,
 627: 4763,
 628: 2487,
 630: 370,
 632: 626,
 633: 2996,
 634: 2740,
 639: 3982,
 640: 1911,
 642: 142,
 643: 597,
 649: 545,
 651: 1108,
 653: 2861,
 655: 2154,
 656: 3036,
 657: 4553,
 659: 1007,
 661: 130,
 662: 6626,
 663: 4376,
 664: 2710,
 665: 1493,
 666: 9212,
 670: 2705,
 673: 962,
 674: 3663,
 675: 388,
 676: 3308,
 677: 2088,
 678: 268,
 679: 579,
 680: 1745,
 681: 1028,
 682: 1459,
 683: 407,
 689: 4475,
 690: 2895,
 691: 3710,
 694: 38,
 698: 5000,
 700: 7279,
 701: 5904,
 704: 1100,
 705: 690,
 706: 1813,
 708: 232,
 709: 1256,
 710: 148

In [23]:
#word_counts = doc_text_pairs.flatMap(lambda x: word_count(x[0], x[1]))
#word_counts.take(10)

[('anarchism', (12, 125)),
 ('political', (12, 26)),
 ('philosophy', (12, 6)),
 ('movement', (12, 23)),
 ('sceptical', (12, 1)),
 ('authority', (12, 18)),
 ('rejects', (12, 1)),
 ('involuntary', (12, 1)),
 ('coercive', (12, 2)),
 ('forms', (12, 9))]

In [24]:
#word_counts_stem = doc_text_pairs.flatMap(lambda x: word_count(x[0], x[1],use_stemming=True))
#word_counts_stem.take(10)

[('anarch', (12, 128)),
 ('polit', (12, 28)),
 ('philosophi', (12, 8)),
 ('movement', (12, 35)),
 ('sceptic', (12, 2)),
 ('author', (12, 23)),
 ('reject', (12, 9)),
 ('involuntari', (12, 1)),
 ('coerciv', (12, 2)),
 ('form', (12, 22))]

In [25]:
#create a body index

# time the index creation time
t_start = time()

# word counts map
word_counts = doc_text_pairs.flatMap(lambda x: word_count(x[0], x[1]))



postings = word_counts.groupByKey().mapValues(reduce_word_counts)
# filtering postings and calculate df

## lowered filter for testing #### need to be 50 instead of 1
postings_filtered = postings.filter(lambda x: len(x[1])>1)
w2df = calculate_df(postings_filtered)
w2df_dict = w2df.collectAsMap()
_ = partition_postings_and_write(postings_filtered,"body").collect()


# collect all posting lists locations into one super-set
super_posting_locs = defaultdict(list)
for blob in client.list_blobs(bucket_name, prefix='postings_gcp_body'):
  if not blob.name.endswith("pickle"):
    continue
  with blob.open("rb") as f:
    posting_locs = pickle.load(f)
    for k, v in posting_locs.items():
      super_posting_locs[k].extend(v)

# Create inverted index instance
inverted = InvertedIndex()
# Adding the posting locations dictionary to the inverted index
#add document length
inverted.DL = doclen
inverted.posting_locs = super_posting_locs
# Add the token - df dictionary to the inverted index
inverted.df = w2df_dict
# write the global stats out
inverted.write_index('.', 'index_body')
# upload to gs
index_src = "index_body.pkl"
index_dst = f'gs://{bucket_name}/indexes/{index_src}'
!gsutil cp $index_src $index_dst

index_body_const_time = time() - t_start

Copying file://index_body.pkl [Content-Type=application/octet-stream]...
/ [1 files][  2.0 MiB/  2.0 MiB]                                                
Operation completed over 1 objects/2.0 MiB.                                      


In [26]:
index_body_const_time

37.07197189331055

In [27]:
#create a title index

# time the index creation time
t_start = time()

# word counts map
word_counts = doc_title_pairs.flatMap(lambda x: word_count(x[0], x[1]))
postings = word_counts.groupByKey().mapValues(reduce_word_counts)
# filtering postings and calculate df
postings_filtered = postings.filter(lambda x: len(x[1])>0)
w2df = calculate_df(postings_filtered)
w2df_dict = w2df.collectAsMap()

                                 #postings instead of postings_filtered
    
_ = partition_postings_and_write(postings_filtered,"title").collect()


# collect all posting lists locations into one super-set
super_posting_locs = defaultdict(list)
for blob in client.list_blobs(bucket_name, prefix='postings_gcp_title'):
  if not blob.name.endswith("pickle"):
    continue
  with blob.open("rb") as f:
    posting_locs = pickle.load(f)
    for k, v in posting_locs.items():
      super_posting_locs[k].extend(v)

# Create inverted index instance
inverted = InvertedIndex()

#addition of doc to title
inverted.doc_to_title = doc_to_title.collectAsMap()

# Adding the posting locations dictionary to the inverted index
inverted.posting_locs = super_posting_locs
# Add the token - df dictionary to the inverted index
inverted.df = w2df_dict
# write the global stats out
inverted.write_index('.', 'index_title')
# upload to gs
index_src = "index_title.pkl"
index_dst = f'gs://{bucket_name}/indexes/{index_src}'
!gsutil cp $index_src $index_dst

index_title_const_time = time() - t_start

Copying file://index_title.pkl [Content-Type=application/octet-stream]...
/ [1 files][ 59.8 KiB/ 59.8 KiB]                                                
Operation completed over 1 objects/59.8 KiB.                                     


In [28]:
index_title_const_time

24.51494026184082

In [29]:

#create a body index WITH STEMMING

# time the index creation time
t_start = time()

# word counts map
word_counts = doc_text_pairs.flatMap(lambda x: word_count(x[0], x[1],use_stemming=True))
postings = word_counts.groupByKey().mapValues(reduce_word_counts)
# filtering postings and calculate df

## lowered filter for testing #### need to be 50 instead of 1
postings_filtered = postings.filter(lambda x: len(x[1])>1)
w2df = calculate_df(postings_filtered)
w2df_dict = w2df.collectAsMap()
_ = partition_postings_and_write(postings_filtered,"body_stem").collect()


# collect all posting lists locations into one super-set
super_posting_locs = defaultdict(list)
for blob in client.list_blobs(bucket_name, prefix='postings_gcp_body_stem'):
  if not blob.name.endswith("pickle"):
    continue
  with blob.open("rb") as f:
    posting_locs = pickle.load(f)
    for k, v in posting_locs.items():
      super_posting_locs[k].extend(v)

# Create inverted index instance
inverted = InvertedIndex()
# Adding the posting locations dictionary to the inverted index
#add document length
inverted.DL = doclen
inverted.posting_locs = super_posting_locs
# Add the token - df dictionary to the inverted index
inverted.df = w2df_dict
# write the global stats out
inverted.write_index('.', 'index_body_stem')
# upload to gs
index_src = "index_body_stem.pkl"
index_dst = f'gs://{bucket_name}/indexes/{index_src}'
!gsutil cp $index_src $index_dst

index_body_stem_const_time = time() - t_start



Copying file://index_body_stem.pkl [Content-Type=application/octet-stream]...
/ [1 files][  1.4 MiB/  1.4 MiB]                                                
Operation completed over 1 objects/1.4 MiB.                                      


In [30]:
index_body_stem_const_time

81.6124222278595

In [31]:
#create a title index WITH STEMMING

# time the index creation time
t_start = time()

# word counts map
word_counts = doc_title_pairs.flatMap(lambda x: word_count(x[0], x[1],use_stemming=True))
postings = word_counts.groupByKey().mapValues(reduce_word_counts)
# filtering postings and calculate df
postings_filtered = postings.filter(lambda x: len(x[1])>0)
w2df = calculate_df(postings_filtered)
w2df_dict = w2df.collectAsMap()

                                 #postings instead of postings_filtered
    
_ = partition_postings_and_write(postings_filtered,"title_stem").collect()


# collect all posting lists locations into one super-set
super_posting_locs = defaultdict(list)
for blob in client.list_blobs(bucket_name, prefix='postings_gcp_title_stem'):
  if not blob.name.endswith("pickle"):
    continue
  with blob.open("rb") as f:
    posting_locs = pickle.load(f)
    for k, v in posting_locs.items():
      super_posting_locs[k].extend(v)

# Create inverted index instance
inverted = InvertedIndex()

#addition of doc to title
inverted.doc_to_title = doc_to_title.collectAsMap()

# Adding the posting locations dictionary to the inverted index
inverted.posting_locs = super_posting_locs
# Add the token - df dictionary to the inverted index
inverted.df = w2df_dict
# write the global stats out
inverted.write_index('.', 'index_title_stem')
# upload to gs
index_src = "index_title_stem.pkl"
index_dst = f'gs://{bucket_name}/indexes/{index_src}'
!gsutil cp $index_src $index_dst

index_title_stem_const_time = time() - t_start

Copying file://index_title_stem.pkl [Content-Type=application/octet-stream]...
/ [1 files][ 56.4 KiB/ 56.4 KiB]                                                
Operation completed over 1 objects/56.4 KiB.                                     


# ANCHOR INDEX

In [83]:
doc_anchor_pairs = parquetFile.limit(1000).select("anchor_text", "id").rdd

In [ ]:
word_counts = doc_anchor_pairs.flatMap(lambda x: word_count(x[0], x[1]))
doc_anchor_pairs.take(100)

Putting it all together

In [84]:
#create a ANCHOR TEXT

# time the index creation time
t_start = time()

# word counts map
word_counts = doc_anchor_pairs.flatMap(lambda x: word_count(x[0], x[1]))
postings = word_counts.groupByKey().mapValues(reduce_word_counts)
# filtering postings and calculate df
postings_filtered = postings.filter(lambda x: len(x[1])>0)
w2df = calculate_df(postings_filtered)
w2df_dict = w2df.collectAsMap()

                                 #postings instead of postings_filtered
    
_ = partition_postings_and_write(postings_filtered,"anchor").collect()


# collect all posting lists locations into one super-set
super_posting_locs = defaultdict(list)
for blob in client.list_blobs(bucket_name, prefix='postings_gcp_anchor'):
  if not blob.name.endswith("pickle"):
    continue
  with blob.open("rb") as f:
    posting_locs = pickle.load(f)
    for k, v in posting_locs.items():
      super_posting_locs[k].extend(v)

# Create inverted index instance
inverted = InvertedIndex()

#addition of doc to title
inverted.doc_to_title = doc_to_title.collectAsMap()

# Adding the posting locations dictionary to the inverted index
inverted.posting_locs = super_posting_locs
# Add the token - df dictionary to the inverted index
inverted.df = w2df_dict
# write the global stats out
inverted.write_index('.', 'index_anchor')
# upload to gs
index_src = "index_anchor.pkl"
index_dst = f'gs://{bucket_name}/indexes/{index_src}'
!gsutil cp $index_src $index_dst

index_anchor_const_time = time() - t_start

23/01/08 14:54:27 WARN org.apache.spark.scheduler.TaskSetManager: Lost task 0.0 in stage 2447.0 (TID 29067) (cluster-d716-w-1.c.ir-project-372716.internal executor 5): org.apache.spark.api.python.PythonException: Traceback (most recent call last):
  File "/usr/lib/spark/python/lib/pyspark.zip/pyspark/worker.py", line 604, in main
    process()
  File "/usr/lib/spark/python/lib/pyspark.zip/pyspark/worker.py", line 594, in process
    out_iter = func(split_index, iterator)
  File "/usr/lib/spark/python/pyspark/rdd.py", line 2916, in pipeline_func
    return func(split, prev_func(split, iterator))
  File "/usr/lib/spark/python/pyspark/rdd.py", line 2916, in pipeline_func
    return func(split, prev_func(split, iterator))
  File "/usr/lib/spark/python/pyspark/rdd.py", line 418, in func
    return f(iterator)
  File "/usr/lib/spark/python/pyspark/rdd.py", line 2234, in combine
    merger.mergeValues(iterator)
  File "/usr/lib/spark/python/lib/pyspark.zip/pyspark/shuffle.py", line 240, in me

Py4JJavaError: An error occurred while calling z:org.apache.spark.api.python.PythonRDD.collectAndServe.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 0 in stage 2447.0 failed 4 times, most recent failure: Lost task 0.3 in stage 2447.0 (TID 29070) (cluster-d716-w-1.c.ir-project-372716.internal executor 6): org.apache.spark.api.python.PythonException: Traceback (most recent call last):
  File "/usr/lib/spark/python/lib/pyspark.zip/pyspark/worker.py", line 604, in main
    process()
  File "/usr/lib/spark/python/lib/pyspark.zip/pyspark/worker.py", line 594, in process
    out_iter = func(split_index, iterator)
  File "/usr/lib/spark/python/pyspark/rdd.py", line 2916, in pipeline_func
    return func(split, prev_func(split, iterator))
  File "/usr/lib/spark/python/pyspark/rdd.py", line 2916, in pipeline_func
    return func(split, prev_func(split, iterator))
  File "/usr/lib/spark/python/pyspark/rdd.py", line 418, in func
    return f(iterator)
  File "/usr/lib/

In [ ]:
index_anchor_const_time

# PageRank

**YOUR TASK (10 POINTS):** Compute PageRank for the entire English Wikipedia. Use your implementation for `generate_graph` function from Colab below.

In [33]:
# Put your `generate_graph` function here
def generate_graph(pages):
  ''' Compute the directed graph generated by wiki links.
  Parameters:
  -----------
    pages: RDD
      An RDD where each row consists of one wikipedia articles with 'id' and 
      'anchor_text'.
  Returns:
  --------
    edges: RDD
      An RDD where each row represents an edge in the directed graph created by
      the wikipedia links. The first entry should the source page id and the 
      second entry is the destination page id. No duplicates should be present. 
    vertices: RDD
      An RDD where each row represents a vetrix (node) in the directed graph 
      created by the wikipedia links. No duplicates should be present. 
  '''
  # YOUR CODE HERE
  edges_id = pages.map(lambda x:(x[0],[i[0] for i in x[1]]))
  edges = edges_id.flatMapValues(lambda x:x).distinct()
  vertices = edges.flatMap(lambda x: (x[0],x[1])).distinct().map(lambda x:([x]))
  return edges, vertices

In [76]:
t_start = time()
#pages_links = spark.read.parquet("gs://wikidata_preprocessed/*").select("id", "anchor_text").rdd
pages_links = parquetFile.limit(1000).select("id", "anchor_text").rdd
# construct the graph 
edges, vertices = generate_graph(pages_links)
# compute PageRank
edgesDF = edges.toDF(['src', 'dst']).repartition(124, 'src')
verticesDF = vertices.toDF(['id']).repartition(124, 'id')
g = GraphFrame(verticesDF, edgesDF)
pr_results = g.pageRank(resetProbability=0.15, maxIter=6)
pr = pr_results.vertices.select("id", "pagerank")
pr = pr.sort(col('pagerank').desc())
#pr.repartition(1).write.csv(f'gs://{bucket_name}/pr', compression="gzip")
pr.repartition(1).toPandas().to_pickle('pagerank.pkl')
index_src = "pagerank.pkl"
index_dst = f'gs://{bucket_name}/pagerank_and_pageview/{index_src}'
!gsutil cp $index_src $index_dst
pr_time = time() - t_start
#pr.show()

Copying file://pagerank.pkl [Content-Type=application/octet-stream]...
/ [1 files][  1.8 MiB/  1.8 MiB]                                                
Operation completed over 1 objects/1.8 MiB.                                      


In [77]:
# test that PageRank computaion took less than 1 hour
pr_time 

85.04862785339355

In [78]:
import requests
r = requests.post('http://34.135.82.254:8080/get_pagerank', json=[606848,39,336])
r.content

b'[\n  1.976891139475762, \n  1.0026870119555944, \n  0.9964858834105783\n]\n'

In [ ]:
# Paths
# Using user page views (as opposed to spiders and automated traffic) for the 
# month of August 2021
t_start = time()
pv_path = 'https://dumps.wikimedia.org/other/pageview_complete/monthly/2021/2021-08/pageviews-202108-user.bz2'
p = Path(pv_path) 
pv_name = p.name
pv_temp = f'{p.stem}-4dedup.txt'
pv_clean = f'{p.stem}.pkl'
# Download the file (2.3GB) 
!wget -N $pv_path
# Filter for English pages, and keep just two fields: article ID (3) and monthly 
# total number of page views (5). Then, remove lines with article id or page 
# view values that are not a sequence of digits.
!bzcat $pv_name | grep "^en\.wikipedia" | cut -d' ' -f3,5 | grep -P "^\d+\s\d+$" > $pv_temp
# Create a Counter (dictionary) that sums up the pages views for the same 
# article, resulting in a mapping from article id to total page views.
wid2pv = Counter()
with open(pv_temp, 'rt') as f:
  for line in f:
    parts = line.split(' ')
    wid2pv.update({int(parts[0]): int(parts[1])})
# write out the counter as binary file (pickle it)
with open(pv_clean, 'wb') as f:
  pickle.dump(wid2pv, f)
pv_time = time() - t_start
# read in the counter
# with open(pv_clean, 'rb') as f:
#   wid2pv = pickle.loads(f.read())

--2023-01-08 12:51:00--  https://dumps.wikimedia.org/other/pageview_complete/monthly/2021/2021-08/pageviews-202108-user.bz2
Resolving dumps.wikimedia.org (dumps.wikimedia.org)... 208.80.154.142, 2620:0:861:2:208:80:154:142
Connecting to dumps.wikimedia.org (dumps.wikimedia.org)|208.80.154.142|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2503235912 (2.3G) [application/octet-stream]
Saving to: ‘pageviews-202108-user.bz2’

pageviews-202108-us 100%[===================>]   2.33G  4.35MB/s    in 8m 54s  

2023-01-08 12:59:55 (4.47 MB/s) - ‘pageviews-202108-user.bz2’ saved [2503235912/2503235912]



In [ ]:
pv_time

978.6952855587006

In [75]:
# copy pageviews to bucket
index_src = "pageviews-202108-user.pkl"
name = "pageview.pkl"
index_dst = f'gs://{bucket_name}/pagerank_and_pageview/{name}'
!gsutil cp $index_src $index_dst

Copying file://pageviews-202108-user.pkl [Content-Type=application/octet-stream]...
/ [1 files][ 73.5 MiB/ 73.5 MiB]                                                
Operation completed over 1 objects/73.5 MiB.                                     


In [82]:
import requests
r = requests.post('http://34.135.82.254:8080/get_pageview', json=[12,39,336])
r.content

b'[\n  75195, \n  24532, \n  25818\n]\n'

# Reporting

**YOUR TASK (5 points):** execute and complete the following lines to complete 
the reporting requirements for assignment #3. 

In [ ]:
# size of input data
!gsutil du -sh "gs://wikidata_preprocessed/"

In [ ]:
# size of index data
index_dst = f'gs://{bucket_name}/postings_gcp/'
!gsutil du -sh "$index_dst"

In [ ]:
# How many USD credits did you use in GCP during the course of this assignment?
cost = 30.04
print(f'I used {cost} USD credit during the course of this assignment')

**Bonus (10 points)** if you implement PageRank in pure PySpark, i.e. without using the GraphFrames package, AND manage to complete 10 iterations of your algorithm on the entire English Wikipedia in less than an hour. 


In [ ]:
#If you have decided to do the bonus task - please copy the code here 

bonus_flag = False # Turn flag on (True) if you have implemented this part

t_start = time()

# PLACE YOUR CODE HERE

pr_time_Bonus = time() - t_start


In [ ]:
# Note:test that PageRank computaion took less than 1 hour
assert pr_time_Bonus < 60*60 and bonus_flag